- 분류를 보고 행렬제곱을 사용해야 한다는 짐작밖에 해내지 못했다.
- 인접 행렬을 제곱한다는 것은 무엇을 의미할까?

### 인접 행렬의 제곱
- 결론만 말하면, 행렬을 N번 제곱한 결과의 행렬은 `간선을 N개 거쳐서 가는 경우의 수`를 담고 있게 된다.

### 14289 본대산책3
- 간선을 N번 거쳐서 0번 노드로 돌아오는 경우의 수를 구하는 문제.
- 그래프 연결정보를 인접행렬로 만든 후 N번 제곱 한다.

In [ ]:
N, M = map(int, input().split())
G = [[0] * N for _ in range(N)]
for _ in range(M) :
  a, b = map(int, input().split())
  G[a-1][b-1] = 1
  G[b-1][a-1] = 1

B = int(input())

def calc(a, a_) :
  res = 0
  for i, v in enumerate(a) :
    res += v * a_[i]
  return res % 1000000007

def power(A) :
  A_ = list(zip(*A))
  AA = [[0] * N for i in range(N)]
  for i, l in enumerate(AA) : 
    for j, v in enumerate(l) :
      l[j] = calc(A[i], A_[j])
  return AA

dp = [G]
answer = [[0]*i+[1]+[0]*(N-i-1) for i in range(N)]
for b in range(B.bit_length()) :
  A = dp[-1]
  dp.append(power(A))
  if B&(1<<b) : #해당 비트가 1이면
    tmp = [[0] * N for _ in range(N)]
    for i, l in enumerate(tmp) :
      for j, v in enumerate(l) :
        l[j] = calc(answer[i], list(zip(*A))[j])
    answer = tmp

print(answer[0][0])

### 가중치
- 인접 행렬의 제곱으로 간선을 거쳐가는 수를 구하고자 한다면, 인접행렬의 가중치는 1로 구성돼있어야 한다.
- 하지만 문제 간선의 가중치는 5이하로 돼있다. 따라서 이 가중치를 1로 바꿔서 생각할 필요가 있다.

### 정점 분할
- 네트워크 플로우에서 자주 사용하는 기법이라고 한다. 미리 알아놔서 나쁠건 없을 것.
- 정점분할 하는 방법은 다음과 같다.
    - 간선이 연결되어있음은, 해당 간선을 이동하는데 1분이 걸리는 것이라고 가정한다.
    1. 가중치가 5 이하이므로 각 정점을 5개로 나눈다.
    2. i번 정점에 대해서 `i, i-1, i-2, i-3, i-4`로 나누고, `i-4` -> `i-3`, `i-3` -> `i-2` .. `i-1` -> `i` 이런식으로 연결한다.
        - 이를 통해 최대 5분까지 걸릴 수 있는 가중치를 1분 걸리는 연결 정보로 바꿀 수 있다.
    3. 분할된 정점에 맞게 연결정보를 입력한다.
        - 예를들어 `u`에서 `v`로 가는데에 3분이 걸린다면, `u` -> `v-2`로 연결하는 식이다.
    4. 결과적으로 전체 그래프의 정점 번호는 `5i, 5i-1, 5i-2, 5i-3, 5i-4`로 구성되게 된다.
        - 직관적으로 생각해서, 각 정점 사이를 5칸으로 쪼갠것이라고 생각하면 될 것 같다.
        - 추가된(사이에 낀) 정점들은 갈 수 있는 방향이 하나 뿐이므로, 자연스럽게 다음 가중치에 반영된다.

In [ ]:
import sys
input=sys.stdin.readline
_N, S, E, T = map(int, input().split())
N = _N * 5
G = [[0] * (N) for _ in range(N)]

for i in range(0, N, 5) :
  G[i+4][i+3] = 1
  G[i+3][i+2] = 1
  G[i+2][i+1] = 1
  G[i+1][i] = 1

for i in range(_N) :
  for j, v in enumerate(input().rstrip()) :
    if v == '0': continue
    v = int(v)
    G[5*i][j*5+v-1] = 1

def calc(a, a_) :
  res = 0
  for i, v in enumerate(a) :
    res += v * a_[i]
  return res % 1_000_003

def power(A) :
  A_ = [*zip(*A)]
  AA = [[0] * N for _ in range(N)]
  for i, l in enumerate(AA) : 
    for j, v in enumerate(l) :
      l[j] = calc(A[i], A_[j])
  return AA

dp = [G]
answer = [[0]*i+[1]+[0]*(N-i-1) for i in range(N)]
for b in range(T.bit_length()) :
  A = dp[-1]
  dp.append(power(A))
  if T&(1<<b) :
    tmp = [[0] * N for _ in range(N)]
    for i, l in enumerate(tmp) :
      for j, v in enumerate(l) :
        l[j] = calc(answer[i], list(zip(*A))[j])
    answer = tmp

print(answer[5*(S-1)][5*(E-1)] % 1_000_003)

행렬제곱 템플릿이 많이 느린것 같다. 손을 봐줘야겠다.